In [115]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [158]:
import csv
import json
import pandas as pd
import numpy as np
from datetime import datetime
import re

## Speeches URLs¶

In [7]:
web = "https://www.presidency.ucsb.edu"
campaignroot0 = "/documents/app-categories/elections-and-transitions/convention-speeches"
campaignrootpage = requests.get(web + campaignroot0)

In [8]:
campaignrootsoup = BeautifulSoup(campaignrootpage.text, "html.parser")

In [9]:
campaignroot = []
campaignroot.append(campaignroot0)
bottom = campaignrootsoup.findAll('ul', attrs = {'class':'pagination'})[0].findAll('a')

while bottom[-2].text == 'next ›':
    nexturl = bottom[-2]['href']
    campaignroot.append(nexturl)
    page = requests.get(web + nexturl)
    soup = BeautifulSoup(page.text,"html.parser")
    bottom = soup.findAll('ul', attrs = {'class':'pagination'})[0].findAll('a')

In [11]:
TextURLs = []
for lk in campaignroot:
    pg = requests.get(web + lk)
    soup = BeautifulSoup(pg.text, "html.parser")
    table = soup.findAll('div', attrs = {'class':'field-title'})
    for tb in table:
        TextURLs.append(tb.a['href'])
    time.sleep(0.05)

## Scrape Speeches

In [12]:
pg = requests.get(web + TextURLs[0])
soup = BeautifulSoup(pg.text, "html.parser")
Content = soup.find('div', attrs = {'class':'field-docs-content'}).text

In [15]:
for i,url in enumerate(TextURLs):
    pg = requests.get(web + url)
    soup = BeautifulSoup(pg.text, "html.parser")
    Title = soup.find('div', attrs={'class':'field-ds-doc-title'}).text
    Time = soup.find('div', attrs = {'class':'field-docs-start-date-time'}).text
    Content = soup.find('div', attrs = {'class':'field-docs-content'}).text
    Name = soup.find('h3', attrs={'class':'diet-title'}).text
    Type = "Convention Speeches"
    time.sleep(0.05)
    if i in list(range(0,3000,100)):
        print(i)
    with open('ConventionSpeech.csv', 'a') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow([Name, Title, Time, Content, Type, url])

0


## Merging with speech data

In [288]:
Speech = pd.read_csv('CampaignSpeech.csv')

In [278]:
Speech.tail()

,Candidate,Title,Date,Content,Type,URL,Year,Month,Day,Speech_index
2734,Franklin D. Roosevelt,"\nAddress at Oglethorpe University in Atlanta,...","\nMay 22, 1932","\nPresident Jacobs, members and friends of Ogl...",Campaign Documents,/documents/address-oglethorpe-university-atlan...,1932,May,22,Speech2734
2735,Franklin D. Roosevelt,"\nAddress at Jefferson Day Dinner in St. Paul,...","\nApril 18, 1932",\nWhat is the real reason that Jefferson Day D...,Campaign Documents,/documents/address-jefferson-day-dinner-st-pau...,1932,April,18,Speech2735
2736,Franklin D. Roosevelt,"\nRadio Address From Albany, New York: ""The 'F...","\nApril 07, 1932",\nAlthough I understand that I am talking unde...,Campaign Documents,/documents/radio-address-from-albany-new-york-...,1932,April,7,Speech2736
2737,Franklin D. Roosevelt,\nThe Governor Enters the First Primary Campai...,"\nJanuary 22, 1932",\nMy dear Mr. McLean:\nIf it is the desire of ...,Campaign Documents,/documents/the-governor-enters-the-first-prima...,1932,January,22,Speech2737
2738,William Howard Taft,\nRemarks Accepting the Presidential Nominatio...,"\nAugust 01, 1912",\nMR. ROOT AND CHAIRMEN OF THE NOTIFICATION CO...,Campaign Documents,/documents/remarks-accepting-the-presidential-...,1912,August,1,Speech2738


In [289]:
ConventionSpeech = pd.read_csv('ConventionSpeech.csv', header=None)

In [290]:
ConventionSpeech.columns = ['Candidate','Title','Date','Content','Type','URL']
ConventionSpeech.head()

,Candidate,Title,Date,Content,Type,URL
0,Hillary Clinton,\nAddress Accepting the Presidential Nominatio...,"\nJuly 28, 2016","\nThank you all very, very much! Thank you for...",Convention Speeches,/documents/address-accepting-the-presidential-...
1,Donald J. Trump,\nAddress Accepting the Presidential Nominatio...,"\nJuly 21, 2016","\nFriends, delegates and fellow Americans: I h...",Convention Speeches,/documents/address-accepting-the-presidential-...
2,Barack Obama,\nRemarks Accepting the Presidential Nominatio...,"\nSeptember 06, 2012",\nThe First Lady. Thank you so much. Tonight I...,Convention Speeches,/documents/remarks-accepting-the-presidential-...
3,Mitt Romney,\nAddress Accepting the Presidential Nominatio...,"\nAugust 30, 2012","\nMr. Chairman, delegates. I accept your nomin...",Convention Speeches,/documents/address-accepting-the-presidential-...
4,John McCain,\nAddress Accepting the Presidential Nominatio...,"\nSeptember 04, 2008",\nThank you. Thank you all very much. Thank yo...,Convention Speeches,/documents/address-accepting-the-presidential-...


In [291]:
ConventionSpeech['Year'] = [int(re.split(',',x)[1].strip()) for x in ConventionSpeech.Date]
ConventionSpeech['Month'] = [re.split(',',x)[0].strip('\n')[:-3] for x in ConventionSpeech.Date]
ConventionSpeech['Day'] = [int(re.split(',',x)[0].strip('\n')[-2:]) for x in ConventionSpeech.Date]

In [292]:
ConventionSpeech = ConventionSpeech[(ConventionSpeech.Year < 1952) | (ConventionSpeech.Year > 1996)]

In [293]:
ConventionSpeech['Speech_index']= ['Speech'+ str(x) for x in range(len(Speech.Title), len(Speech.Title) + len(ConventionSpeech.Title))]

In [294]:
Speech = Speech.append(ConventionSpeech)

In [295]:
Speech.to_csv("CampaignSpeech2.csv", index = False)

In [299]:
ConventionSpeech.to_csv("ConventionSpeech2.csv", index=False)

## Merge at Subspeech Level

In [262]:
sub_dict = {'Subspeech_index':[], 'SubContent':[], 'Speech_index':[]}

In [300]:
ConventionSpeech = pd.read_csv("ConventionSpeech2.csv")

In [302]:
# Save every 10 lines into a subspeech e.g. [0,10)

for i,text in enumerate(ConventionSpeech.Content): 
    splited = re.split('\n',text)
    length = len(re.split('\n',text))
    if length > 10:
        a = list(range(0,length,10)) 
        for j in a[:-2]:
            rowvalue = []
            upper=j+10
            sub_dict['Subspeech_index'].append(ConventionSpeech['Speech_index'][i]+'_'+str(j)+ ':' + str(upper))
            sub_dict['SubContent'].append(splited[j: upper])
            sub_dict['Speech_index'].append(ConventionSpeech['Speech_index'][i])
            
        sub_dict['Subspeech_index'].append(ConventionSpeech['Speech_index'][i]+'_'+ str(a[-2])+ ':' + str(length))
        sub_dict['SubContent'].append(splited[a[-2]: length])
        sub_dict['Speech_index'].append(ConventionSpeech['Speech_index'][i])
    else:
        sub_dict['Subspeech_index'].append(ConventionSpeech['Speech_index'][i] +'_'+str(0)+ ':' + str(length))
        sub_dict['SubContent'].append(splited[: length])
        sub_dict['Speech_index'].append(ConventionSpeech['Speech_index'][i])

In [303]:
Sub_Speech = pd.DataFrame.from_dict(sub_dict)
len(Sub_Speech.Speech_index.unique())

25

In [304]:
SubSpeech = Sub_Speech.merge(ConventionSpeech, left_on='Speech_index', right_on = 'Speech_index')
SubSpeech.tail()

,Subspeech_index,SubContent,Speech_index,Candidate,Title,Date,Content,Type,URL,Year,Month,Day
924,Speech2762_60:70,[The late panic disclosed a lack of elasticity...,Speech2762,William Howard Taft,\nAddress Accepting the Republican Presidentia...,"\nJuly 28, 1908",\nSenator Warner and Gentlemen of the Committe...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1908,July,28
925,Speech2762_70:80,[The proposition of the Democratic platform is...,Speech2762,William Howard Taft,\nAddress Accepting the Republican Presidentia...,"\nJuly 28, 1908",\nSenator Warner and Gentlemen of the Committe...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1908,July,28
926,Speech2762_80:99,[I have long been of the opinion that the vari...,Speech2762,William Howard Taft,\nAddress Accepting the Republican Presidentia...,"\nJuly 28, 1908",\nSenator Warner and Gentlemen of the Committe...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1908,July,28
927,Speech2763_0:10,"[, Senator Lodge and Gentlemen of the Notifica...",Speech2763,William McKinley,\nAddress Accepting the Republican Presidentia...,"\nJuly 12, 1900",\nSenator Lodge and Gentlemen of the Notificat...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1900,July,12
928,Speech2763_10:28,"[The National credit, which has thus far fortu...",Speech2763,William McKinley,\nAddress Accepting the Republican Presidentia...,"\nJuly 12, 1900",\nSenator Lodge and Gentlemen of the Notificat...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1900,July,12


In [305]:
Sub_Speech_old = pd.read_csv('SubSpeech.csv')

In [306]:
Sub_Speech_old.drop(columns=['Unnamed: 0'], inplace=True)

In [307]:
Sub_Speech_new = Sub_Speech_old.append(SubSpeech)

In [309]:
Sub_Speech_new.tail()

,Subspeech_index,SubContent,Speech_index,Candidate,Title,Date,Content,Type,URL,Year,Month,Day
924,Speech2762_60:70,[The late panic disclosed a lack of elasticity...,Speech2762,William Howard Taft,\nAddress Accepting the Republican Presidentia...,"\nJuly 28, 1908",\nSenator Warner and Gentlemen of the Committe...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1908,July,28
925,Speech2762_70:80,[The proposition of the Democratic platform is...,Speech2762,William Howard Taft,\nAddress Accepting the Republican Presidentia...,"\nJuly 28, 1908",\nSenator Warner and Gentlemen of the Committe...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1908,July,28
926,Speech2762_80:99,[I have long been of the opinion that the vari...,Speech2762,William Howard Taft,\nAddress Accepting the Republican Presidentia...,"\nJuly 28, 1908",\nSenator Warner and Gentlemen of the Committe...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1908,July,28
927,Speech2763_0:10,"[, Senator Lodge and Gentlemen of the Notifica...",Speech2763,William McKinley,\nAddress Accepting the Republican Presidentia...,"\nJuly 12, 1900",\nSenator Lodge and Gentlemen of the Notificat...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1900,July,12
928,Speech2763_10:28,"[The National credit, which has thus far fortu...",Speech2763,William McKinley,\nAddress Accepting the Republican Presidentia...,"\nJuly 12, 1900",\nSenator Lodge and Gentlemen of the Notificat...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1900,July,12


In [308]:
Sub_Speech_new.to_csv('SubSpeech2.csv', index=False)

## Add Penn data

In [506]:
with open('apda.json') as penn_json:
    PennSpeech_dict = json.load(penn_json)

In [133]:
#PennSpeech_dict.keys()

In [507]:
PennSpeech = pd.DataFrame.from_dict(PennSpeech_dict, orient='index')

In [359]:
#PennSpeech.head()

In [508]:
PennSpeech.Date.replace([-999], np.nan, inplace = True)

In [509]:
PennSpeech = PennSpeech[PennSpeech.Genre == 'SPEECH']

In [510]:
Month = []
Day = []

for x in PennSpeech.Date:
    if type(x) == str:
        Month.append(int(re.split('/',x)[0]))
        Day.append(int(re.split('/',x)[-1]))            
    else:
        Month.append(x)
        Day.append(x)

In [511]:
PennSpeech['Month'] = Month
PennSpeech['Day'] = Day

In [512]:
PennSpeech.rename(columns = {'Type':'Chanel', 'Genre':'Type', 'Text':'Content'}, inplace=True)

In [513]:
#Add speech index
PennSpeech['Speech_index'] = ['Speech'+ str(x) for x in range(len(Speech.Year), len(Speech.Year) + len(PennSpeech.Year))]


In [514]:
PennSpeech.drop(columns=['Party', 'Length'], inplace=True)

In [515]:
PennSpeech['Title'] = np.nan
PennSpeech['URL'] = np.nan

In [518]:
##recode candidate names 
PennSpeech.Candidate.replace(['STEVENSON'],'Adlai Stevenson', inplace=True)
PennSpeech.Candidate.replace(['KENNEDY'],'John F. Kennedy', inplace=True)
PennSpeech.Candidate.replace(['NIXON'],'Richard Nixon', inplace=True)
PennSpeech.Candidate.replace(['JOHNSON'],'Lyndon B. Johnson', inplace=True)
PennSpeech.Candidate.replace(['HUMPHREY'],'Hubert H. Humphrey', inplace=True)

PennSpeech.Candidate.replace(['MCGOVERN'],'George McGovern', inplace=True)
PennSpeech.Candidate.replace(['CARTER'],'Jimmy Carter', inplace=True)
PennSpeech.Candidate.replace(['FORD'],'Gerald R. Ford', inplace=True)
PennSpeech.Candidate.replace(['REAGAN'],'Ronald Reagan', inplace=True)
PennSpeech.Candidate.replace(['EISENHOWER'],'Dwight Eisenhower', inplace=True)

PennSpeech.Candidate.replace(['MONDALE'],'Walter F. Mondale', inplace=True)
PennSpeech.Candidate.replace(['DUKAKIS'],'Michael S. Dukakis', inplace=True)
PennSpeech.Candidate.replace(['BUSH'],'George Bush', inplace=True)
PennSpeech.Candidate.replace(['CLINTON'],'William J. Clinton', inplace=True)
PennSpeech.Candidate.replace(['DOLE'],'Robert Dole', inplace=True)

In [519]:
Speech['City'] = np.nan
Speech['State'] = np.nan
Speech['Description'] = np.nan
Speech['Subject'] = np.nan

In [520]:
Speech_merged = Speech.append(PennSpeech)

In [367]:
#Speech_merged.tail()

In [580]:
PennSpeech.Candidate.value_counts()

John F. Kennedy       312
Dwight Eisenhower     273
Adlai Stevenson       261
Richard Nixon         253
William J. Clinton    187
Ronald Reagan         178
George Bush           167
Hubert H. Humphrey    153
Jimmy Carter          144
Gerald R. Ford        128
Lyndon B. Johnson     111
George McGovern        98
Robert Dole            78
Michael S. Dukakis     66
Walter F. Mondale      66
Name: Candidate, dtype: int64

In [521]:
PennSpeech.to_csv("PennSpeechCleaned.csv", index = False)

In [522]:
Speech_merged.to_csv("CampaignSpeechMerged.csv", index=False)

### create subspeech and merge 

In [523]:
PennSpeech = pd.read_csv('PennSpeechCleaned.csv')

In [371]:
#PennSpeech.head()

In [524]:
sub_dict = {'Subspeech_index':[], 'SubContent':[], 'Speech_index':[]}

In [525]:
# Save every 10 paragraphs into a subspeech e.g. [0,8)

for i,text in enumerate(PennSpeech['Content']): 
    splited = re.split('\n\n',text)
    length = len(re.split('\n\n',text))
    if length > 10:
        a = list(range(0,length,10)) 
        for j in a[:-2]:
            rowvalue = []
            upper=j+10
            sub_dict['Subspeech_index'].append(PennSpeech['Speech_index'][i]+'_'+str(j)+ ':' + str(upper))
            sub_dict['SubContent'].append(splited[j: upper])
            sub_dict['Speech_index'].append(PennSpeech['Speech_index'][i])
            
        sub_dict['Subspeech_index'].append(PennSpeech['Speech_index'][i]+'_'+ str(a[-2])+ ':' + str(length))
        sub_dict['SubContent'].append(splited[a[-2]: length])
        sub_dict['Speech_index'].append(PennSpeech['Speech_index'][i])
    else:
        sub_dict['Subspeech_index'].append(PennSpeech['Speech_index'][i] +'_'+str(0)+ ':' + str(length))
        sub_dict['SubContent'].append(splited[: length])
        sub_dict['Speech_index'].append(PennSpeech['Speech_index'][i])

In [526]:
Sub_Speech = pd.DataFrame.from_dict(sub_dict)
Sub_Speech.head()

,Subspeech_index,SubContent,Speech_index
0,Speech2764_0:10,"[Mr. Tepe, Mr. DiSalle, and distinguished gues...",Speech2764
1,Speech2764_10:20,"[\tThis is my first campaign visit to Ohio, an...",Speech2764
2,Speech2764_20:30,[\tThere is a good reason why voters are suspi...,Speech2764
3,Speech2764_30:48,"[\tWell, we know which way Warren Harding comp...",Speech2764
4,Speech2765_0:16,[Lyndon; Ralph--Senator Yarborough; Mrs. Johns...,Speech2765


In [527]:
SubSpeech_Penn = Sub_Speech.merge(PennSpeech, left_on='Speech_index', right_on = 'Speech_index')

In [424]:
#SubSpeech_Penn.tail()

In [411]:
#Sub_Speech.Speech_index.value_counts()

In [528]:
Sub_Speech2 = pd.read_csv('SubSpeech2.csv')

In [529]:
Sub_Speech2['City'] = np.nan
Sub_Speech2['State'] = np.nan
Sub_Speech2['Description'] = np.nan
Sub_Speech2['Subject'] = np.nan

In [530]:
SubSpeech_Merged = Sub_Speech2.append(SubSpeech_Penn)

In [531]:
SubSpeech_Merged.to_csv('SubSpeech_Merged.csv', index=False)

## Add Annotation

In [532]:
with open('Annotations.jsonl/test_annotation.jsonl', 'r') as annotation_jsonl:
    annotation_list = list(annotation_jsonl)
with open('Annotations.jsonl/pop_code1117.jsonl', 'r') as annotation_jsonl:
    annotation_list += list(annotation_jsonl)
with open('Annotations.jsonl/blind_test1117.jsonl', 'r') as annotation_jsonl:
    annotation_list += list(annotation_jsonl)
with open('Annotations.jsonl/blindtest2.jsonl', 'r') as annotation_jsonl:
    annotation_list += list(annotation_jsonl)

In [533]:
len(annotation_list)

419

In [534]:
coded = {}
for annotation in annotation_list:
    annotation_dict = json.loads(annotation)
    coded[annotation_dict['text'].split('##')[0]] = annotation_dict['answer']

In [535]:
coded_data = pd.DataFrame.from_dict(coded, orient='index')
coded_data = coded_data.rename(columns={0:'Pop'})

In [536]:
coded_data['Subspeech_index'] = coded_data.index

In [537]:
Sub_Speech = pd.read_csv('SubSpeech_Merged.csv')

In [538]:
SubSpeech_coded = Sub_Speech.merge(coded_data, how = 'outer', left_on='Subspeech_index', right_on = 'Subspeech_index')

In [539]:
SubSpeech_coded.to_csv("SubSpeech_mergedcoded.csv", index=False)

## Delete possible duplicats

In [564]:
SubSpeech_coded = pd.read_csv("SubSpeech_mergedcoded.csv")

In [542]:
overmonth = ['September', 'October', 'November']

In [568]:
dup_index = SubSpeech_coded[SubSpeech_coded.Year.isin(PennSpeech.Year) & SubSpeech_coded.Type.isin(['Campaign Documents']) 
                            & SubSpeech_coded.Month.isin(overmonth)].index

In [571]:
coded_index = SubSpeech_coded[SubSpeech_coded.Year.isin(PennSpeech.Year) & 
                SubSpeech_coded.Month.isin(overmonth) & SubSpeech_coded.Type.isin(['Campaign Documents']) 
                              & SubSpeech_coded.Pop.isin(['reject','accept'])].index


In [573]:
del_index = list(set(dup_index).difference(set(coded_index)))

In [575]:
len(del_index)

2383

In [576]:
SubSpeech_coded_cleaned = SubSpeech_coded.drop(del_index)

In [552]:
SubSpeech_coded_cleaned.to_csv('SubSpeech_coded_cleaned.csv', index = False)

In [577]:
len(SubSpeech_coded_cleaned.Speech_index.unique())

4363

In [554]:
Speech_merged = pd.read_csv('CampaignSpeechMerged.csv')

In [557]:
Speech_merged_cleaned = Speech_merged[Speech_merged.Speech_index.isin(SubSpeech_coded_cleaned.Speech_index)]

In [558]:
len(Speech_merged.Speech_index), len(Speech_merged_cleaned.Speech_index)

(5239, 4363)

In [560]:
Speech_merged_cleaned.to_csv('CampaignSpeechMerged_cleaned.csv', index=False)